# Lecture 7: Mapping and nested data frames

### Lecture learning objectives:

By then end of this lecture & worksheet 7, students should be able to:

* Write and use anonymous functions in R in isolation and in combination the functional {purrr} `map`
* (OPTIONAL) Understand the general ideas of the `map*`, `map2*` and `pmap*` variant functions in the {purrr} package and the six types of output options (list, double, integer, logical, character and data frame).
* Create and modify nested data frames using {dplyr} `group_by` + {tidyr} `nest` and {purrr} `map*` functions
* Create unnested data frames using {tidyr} `unnest` 
* Describe situations where nested data frames are useful 

In [29]:
library(gapminder)
library(infer)
library(tidyverse)
options(repr.matrix.max.rows = 10)

## More functional style programming in R with `purrr`


![](https://ih1.redbubble.net/image.329884292.2339/sticker,375x360-bg,ffffff.u1.png)

https://purrr.tidyverse.org/

### Passing arguments to functions you are mapping

- What if you need to pass arguments to the functions you are mapping, in some cases you can take advantage of `...` (e.g., `na.rm = TRUE`) but if things get more complex than that, anonymous functions are your friends. 

- For example, imagine mapping `str_replace` to all the columns of a very wide and nested data frame to replace all instances of `"Cdn"` with `"Canadian"` to fix a data entry error that occurs in several columns in a data set that occur in a nested list column. Here we need to provide the arguments `pattern` and `replacement` in addition to the data we are operating on.

- This is probably the only time I use anonymous functions regularly.

#### Anonymous functions in R

General format: `function(x) body_of_function`
 
 To use one in the global environment, outside of another function call, you do the following:

In [30]:
(function(x) x + 1)(1)

[1] 2

Above the function takes in x as an argument and adds one to it. The function definition is surrounded by round brackets, as is the value being passed to the anonymous function.

#### Back to anonymous function calls within {purrr} `map_*`

Long form:

```
map_*(data, function(arg) function_being_called(arg, other_arg))
```


Short form:

```
map_*(data, ~ function_being_called(., other_arg))
```

In the shortcut we replace `function(arg)` with a `~` and replace the `arg` in the function call with a `.`

#### Example:

Map `str_replace` to all the columns of a very wide data frame to replace all instances of `"Cdn"` with `"Canadian"` to fix a data entry error that occurs in several columns in a data set. 

In [31]:
data_entry <- tibble(id = c("25323", "45234", "23471"),
                    birth_citizenship = c("Canadian", "American", "Cdn"),
                    current_citizenship = c("Canadian", "Vietnamese", "Cdn"))
data_entry

id,birth_citizenship,current_citizenship
<chr>,<chr>,<chr>
25323,Canadian,Canadian
45234,American,Vietnamese
23471,Cdn,Cdn


Using verbose anonymous function syntax:

In [32]:
map_df(data_entry, function(vect) str_replace(vect, pattern = "Cdn", replacement = "Canadian"))

id,birth_citizenship,current_citizenship
<chr>,<chr>,<chr>
25323,Canadian,Canadian
45234,American,Vietnamese
23471,Canadian,Canadian


Using shorthand anonymous function syntax:

In [33]:
map_df(data_entry, ~str_replace(., pattern = "Cdn", replacement = "Canadian"))

id,birth_citizenship,current_citizenship
<chr>,<chr>,<chr>
25323,Canadian,Canadian
45234,American,Vietnamese
23471,Canadian,Canadian


Acknowledgement: here you do not need to use an anonymous function, you can take advantage of `...` but we will see more complicated cases later in the lecture and in the lab. Complicated cases arise when you are working with nested data frames.

In [34]:
map_df(data_entry, str_replace, pattern = "Cdn", replacement = "Canadian")

id,birth_citizenship,current_citizenship
<chr>,<chr>,<chr>
25323,Canadian,Canadian
45234,American,Vietnamese
23471,Canadian,Canadian


## (OPTIONAL) Mapping with > 1 data objects

What if the function you want to map takes in > 1 data objects?

`map2*` and `pmap*` are your friends here!

#### `purrr::map2*`

```
map2*(.x, .y, .f, ...)
```

Above reads as: `for` every element of `.x` and `.y` apply `.f` 

Or picture as...

<img src="img/minis_map2_data.png" width=700>

*Source: [purrr workshop](https://speakerdeck.com/jennybc/purrr-workshop) by Jenny Bryan*

<img src="img/minis_map2.png" width=700>

*Source: [purrr workshop](https://speakerdeck.com/jennybc/purrr-workshop) by Jenny Bryan*

#### `purrr::map2_df` example:


For example, say you want to calculate a weighted means (using `weighted.mean`) for columns of a data frame where you had another data frame containing those weights.

Let's make some data:

In [35]:
data <- tibble(frequency = runif(10),
             loudness = runif(10),
               power = runif(10),
              rating = rpois(10, 5) + 1,
                year = rpois(10, 5) + 1999)
data[1, 1] <- NA
data

frequency,loudness,power,rating,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NA,0.8971602,0.09913619,6,2004
0.07389762,0.9885117,0.13073881,3,2003
0.20319018,0.4464637,0.40757000,5,2002
0.40116953,0.2585920,0.77210928,4,2001
0.61199899,0.2772221,0.50574766,5,2004
0.97175302,0.4691002,0.95197273,7,2001
0.21701388,0.8217019,0.23764613,8,2007
0.33370896,0.8287219,0.20251747,2,2009
0.06301434,0.5701445,0.71843132,8,2005


In [36]:
library(dplyr, quietly = TRUE)
data <- tibble(x1 = runif(10),
               x2 = runif(10),
               x3 = runif(10))
data[1, 1] <- NA
weights <- tibble(x1 = rpois(10, 5) + 1,
                 x2 = rpois(10, 5) + 1,
                 x3 = rpois(10, 5) + 1,)

data
weights

x1,x2,x3
<dbl>,<dbl>,<dbl>
NA,0.99335250,0.50502241
0.70825947,0.57027714,0.44712861
0.56335713,0.00521319,0.03118581
0.87800334,0.17278128,0.22822493
0.45827125,0.72443217,0.37442013
0.39525061,0.42935012,0.22785951
0.21861254,0.05073587,0.88371480
0.07253473,0.84117550,0.02669344
0.08383891,0.68849135,0.25799151


x1,x2,x3
<dbl>,<dbl>,<dbl>
6,8,6
1,8,3
6,8,5
7,8,5
4,8,4
5,8,6
7,5,5
6,6,7
5,9,8


### `purrr::map2_df` example:

Let's use `map2_df` to calculate the weighted mean using these two data frames.

In [37]:
?weighted.mean

weighted.mean {stats},R Documentation
x,an object containing the values whose weighted mean is to be computed.
w,a numerical vector of weights the same length as x giving the weights to use for elements of x.
...,arguments to be passed to or from methods.
na.rm,a logical value indicating whether NA values in x should be stripped before the computation proceeds.


Now that we know how to use `weighted.mean`, let's map it! (and use `na.rm = TRUE` to deal with NA's)

In [38]:
map2_df(data, weights, weighted.mean, na.rm = TRUE)

x1,x2,x3
<dbl>,<dbl>,<dbl>
0.5145843,0.5210375,0.328147


Not too bad eh! 

#### `purrr::map2*`

Also, if `y` has less elements than `x`, it recycles `y`:

<img src="https://d33wubrfki0l68.cloudfront.net/55032525ec77409e381dcd200a47e1787e65b964/dcaef/diagrams/functionals/map2-recycle.png" width=400>

This is most useful when y has only one element.

#### `purrr::pmap*`

```
pmap*(list(.x1, .x2, ... .xn), .f, ...)
```

Above reads as: `for` every element of in the **list** (that contains `.x1, .x2, ... .xn`) apply `.f` 

But what happens when you have > 2 arguments?

## (OPTIONAL) Mapping more than two arguments

Without an anonymous function, works as so:

In [39]:
f1 <- function(x, y, z) {
    x + y + z
}

pmap_dbl(list(c(1, 1), c(1, 2), c(2, 2)), f1)

[1] 4 5

If you want to use an anonymous function, then use `..1`, `..2`, `..3`, and so on to specify where the mapped objets go in your function:

In [40]:
f2 <- function(x, y, z, a = 0) {
    x + y + z + a
}

pmap_dbl(list(c(1, 1), c(1, 2), c(2, 2)), ~ f2(..1, ..2, ..3, a = -1))

[1] 3 4

We only used two inputs to our function here, but we can use any number with `pmap`, we just need to add them to our list!

### Want to iterate row-wise, instead of column-wise?

Here you can use `purrr::pmap` on a single data frame!

This: ```purrr::pmap(df, .f)```

reads as: `for` every tuple in `.l` (*i.e.*, each row of `df`) apply `.f`

The key point is that `pmap()` iterates over tuples = the collection of `i`-th elements of `k` lists. A data frame row is an interesting special case.

### Here's an example of row-wise iteration 

Here we calculate the sum for each row in the `mtcars` data frame:

In [41]:
pmap(mtcars, sum)

[[1]]
[1] 328.98

[[2]]
[1] 329.795

[[3]]
[1] 259.58

[[4]]
[1] 426.135

[[5]]
[1] 590.31

[[6]]
[1] 385.54

[[7]]
[1] 656.92

[[8]]
[1] 270.98

[[9]]
[1] 299.57

[[10]]
[1] 350.46

[[11]]
[1] 349.66

[[12]]
[1] 510.74

[[13]]
[1] 511.5

[[14]]
[1] 509.85

[[15]]
[1] 728.56

[[16]]
[1] 726.644

[[17]]
[1] 725.695

[[18]]
[1] 213.85

[[19]]
[1] 195.165

[[20]]
[1] 206.955

[[21]]
[1] 273.775

[[22]]
[1] 519.65

[[23]]
[1] 506.085

[[24]]
[1] 646.28

[[25]]
[1] 631.175

[[26]]
[1] 208.215

[[27]]
[1] 272.57

[[28]]
[1] 273.683

[[29]]
[1] 670.69

[[30]]
[1] 379.59

[[31]]
[1] 694.71

[[32]]
[1] 288.89

### Mapping over groups of rows

There are two strategies, we have already learned about the first:
- `dplyr::group_by` + `dplyr::summarize`
- `dplyr::group_by` + `tidyr::nest`

## Nested data frames

In future stastistics classes, we will use nested data frames for working with models and estimates. For example, in DSCI 552 we will use them for calculating confidence intervals. Here is some sample code below to calculcate confidence intervals for the mean life expectency for each each country in the `gapminder` data set:

In [42]:
gap_lifeExp_ci <- function(df, statistic) {
  df %>% 
        specify(response = lifeExp) %>% 
        generate(reps = 1000, type = "bootstrap")  %>% 
        calculate(stat = statistic)  %>% 
        get_ci()
}

by_country <- gapminder %>%
    group_by(continent, country) %>%
    nest() %>% 
    mutate(mean_life_exp = map_dbl(data, ~mean(.$lifeExp)), 
    life_exp_ci = map(data, ~gap_lifeExp_ci(., "mean")))
print(by_country)

# A tibble: 142 × 5
# Groups:   continent, country [142]
   country     continent data              mean_life_exp life_exp_ci     
   <fct>       <fct>     <list>                    <dbl> <list>          
 1 Afghanistan Asia      <tibble [12 × 4]>          37.5 <tibble [1 × 2]>
 2 Albania     Europe    <tibble [12 × 4]>          68.4 <tibble [1 × 2]>
 3 Algeria     Africa    <tibble [12 × 4]>          59.0 <tibble [1 × 2]>
 4 Angola      Africa    <tibble [12 × 4]>          37.9 <tibble [1 × 2]>
 5 Argentina   Americas  <tibble [12 × 4]>          69.1 <tibble [1 × 2]>
 6 Australia   Oceania   <tibble [12 × 4]>          74.7 <tibble [1 × 2]>
 7 Austria     Europe    <tibble [12 × 4]>          73.1 <tibble [1 × 2]>
 8 Bahrain     Asia      <tibble [12 × 4]>          65.6 <tibble [1 × 2]>
 9 Bangladesh  Asia      <tibble [12 × 4]>          49.8 <tibble [1 × 2]>
10 Belgium     Europe    <tibble [12 × 4]>          73.6 <tibble [1 × 2]>
# … with 132 more rows


This is called a nested data frame. A data frame which contains other data frames. This data structure is extremely useful when fitting many models and allows you to keep the data, the model meta data, and the model and its results all associated together as a single row in a data frame.

Let's unpack this data structure a bit more and learn how to create, manipulate them.

### List-columns

To create a nested data frame we start with a **grouped** data frame, and “nest” it:

In [43]:
# create a nested data frame
by_country <- gapminder %>% 
    group_by(continent, country) %>% 
    nest()
print(by_country)

# A tibble: 142 × 3
# Groups:   continent, country [142]
   country     continent data             
   <fct>       <fct>     <list>           
 1 Afghanistan Asia      <tibble [12 × 4]>
 2 Albania     Europe    <tibble [12 × 4]>
 3 Algeria     Africa    <tibble [12 × 4]>
 4 Angola      Africa    <tibble [12 × 4]>
 5 Argentina   Americas  <tibble [12 × 4]>
 6 Australia   Oceania   <tibble [12 × 4]>
 7 Austria     Europe    <tibble [12 × 4]>
 8 Bahrain     Asia      <tibble [12 × 4]>
 9 Bangladesh  Asia      <tibble [12 × 4]>
10 Belgium     Europe    <tibble [12 × 4]>
# … with 132 more rows


(I’m cheating a little by grouping on both continent and country. Given country, continent is fixed, so this doesn’t add any more groups, but it’s an easy way to carry an extra variable along for the ride.)

What is the `data` column here? 

In [44]:
class(by_country$data)

[1] "list"

The `data` column is actually a list of data frames (or tibbles, to be precise). This seems like a crazy idea: we have a data frame with a column that is a list of other data frames! 

Let's look at what the first element of the `data` list column looks like:

In [45]:
# look at first element of the data list column
by_country$data[[1]]

year,lifeExp,pop,gdpPercap
<int>,<dbl>,<int>,<dbl>
1952,28.801,8425333,779.4453
1957,30.332,9240934,820.8530
1962,31.997,10267083,853.1007
1967,34.020,11537966,836.1971
1972,36.088,13079460,739.9811
⋮,⋮,⋮,⋮
1987,40.822,13867957,852.3959
1992,41.674,16317921,649.3414
1997,41.763,22227415,635.3414


Note the difference between a standard grouped data frame and a nested data frame: in a grouped data frame, each row is an observation; in a nested data frame, each row is a group. Another way to think about a nested dataset is we now have a meta-observation: a row that represents the complete time course for a country, rather than a single point in time.

#### (OPTIONAL) List-columns in data frames versus tibbles

List columns are possible in base R data frames, but making them isn't easy and you have to jump through some hoops.

If we try to make a data frame with a single list column, `data.frame` treats the list as a list of columns...



In [46]:
# try to make a list column in a data frame
data.frame(x = list(c(1, 2, 3), c(3, 4, 5)))

x.c.1..2..3.,x.c.3..4..5.
<dbl>,<dbl>
1,3
2,4
3,5


To make the work, we need to use the `I` function (which changes the class of an object to indicate that it should be treated ‘as is’):

In [47]:
# try to make a list column in a data frame by using I
data.frame(x = I(list(c(1, 2, 3), c(3, 4, 5))))

x
<I<list>>
"1, 2, 3"
"3, 4, 5"


Tibble alleviates this problem by being lazier (`tibble` doesn’t modify its inputs) and by providing a better print method:

In [48]:
# try to make a list column in a tibble
tibble(x = list(c(1, 2, 3), c(3, 4, 5)))

x
<list>
"1, 2, 3"
"3, 4, 5"


### List column workflow:

1. Create a list column using a function such as `nest` (other possibilities are `summarise` + `list`, or `mutate` + a `map_*` function, as described in [Creating list-columns](https://r4ds.had.co.nz/many-models.html#creating-list-columns))

2. Create other intermediate list-columns by transforming existing list columns with `map`, `map2` or `pmap`

3. Simplify the list-column back down to a data frame or atomic vector, often by `unnest`, `mutate` + `map_*` functions that return atomic vectors as opposed to lists. 

We've talked about step 1, and what a list column is. Now let's explore how we can create other intermediate list-columns by transforming existing columns with `map`.

### Create other intermediate list-columns with `map`

Remembering our example:

In [49]:
gap_lifeExp_ci <- function(df, statistic) {
  df %>% 
        specify(response = lifeExp) %>% 
        generate(reps = 1000, type = "bootstrap")  %>% 
        calculate(stat = statistic)  %>% 
        get_ci()
}

# create a nested gapminder tibble
by_country <- gapminder %>% 
    group_by(continent, country) %>% 
    nest() 

In [50]:
print(by_country)

# A tibble: 142 × 3
# Groups:   continent, country [142]
   country     continent data             
   <fct>       <fct>     <list>           
 1 Afghanistan Asia      <tibble [12 × 4]>
 2 Albania     Europe    <tibble [12 × 4]>
 3 Algeria     Africa    <tibble [12 × 4]>
 4 Angola      Africa    <tibble [12 × 4]>
 5 Argentina   Americas  <tibble [12 × 4]>
 6 Australia   Oceania   <tibble [12 × 4]>
 7 Austria     Europe    <tibble [12 × 4]>
 8 Bahrain     Asia      <tibble [12 × 4]>
 9 Bangladesh  Asia      <tibble [12 × 4]>
10 Belgium     Europe    <tibble [12 × 4]>
# … with 132 more rows


We'd like to apply the mean function to get the mean life expectancy in a column:

In [51]:
by_country <- gapminder %>%
    group_by(continent, country) %>%
    nest() %>% 
    mutate(mean_life_exp = map_dbl(data, ~ mean(.$lifeExp)))  
print(by_country)

# A tibble: 142 × 4
# Groups:   continent, country [142]
   country     continent data              mean_life_exp
   <fct>       <fct>     <list>                    <dbl>
 1 Afghanistan Asia      <tibble [12 × 4]>          37.5
 2 Albania     Europe    <tibble [12 × 4]>          68.4
 3 Algeria     Africa    <tibble [12 × 4]>          59.0
 4 Angola      Africa    <tibble [12 × 4]>          37.9
 5 Argentina   Americas  <tibble [12 × 4]>          69.1
 6 Australia   Oceania   <tibble [12 × 4]>          74.7
 7 Austria     Europe    <tibble [12 × 4]>          73.1
 8 Bahrain     Asia      <tibble [12 × 4]>          65.6
 9 Bangladesh  Asia      <tibble [12 × 4]>          49.8
10 Belgium     Europe    <tibble [12 × 4]>          73.6
# … with 132 more rows


Now we'd like to apply the `gap_lifeExp_ci` function to each tibble in the `data` list column to obtain another list column containing the confidence interval tibbles. We can use `mutate` + `map` to do this:

In [53]:
gap_lifeExp_ci <- function(df, statistic) {
  df %>% 
        specify(response = lifeExp) %>% 
        generate(reps = 1000, type = "bootstrap")  %>% 
        calculate(stat = statistic)  %>% 
        get_ci()
}

by_country <- gapminder %>%
    group_by(continent, country) %>%
    nest() %>% 
    mutate(mean_life_exp = map_dbl(data, ~mean(.$lifeExp)), 
    life_exp_ci = map(data, ~gap_lifeExp_ci(., "mean")))
print(by_country)

# A tibble: 142 × 5
# Groups:   continent, country [142]
   country     continent data              mean_life_exp life_exp_ci     
   <fct>       <fct>     <list>                    <dbl> <list>          
 1 Afghanistan Asia      <tibble [12 × 4]>          37.5 <tibble [1 × 2]>
 2 Albania     Europe    <tibble [12 × 4]>          68.4 <tibble [1 × 2]>
 3 Algeria     Africa    <tibble [12 × 4]>          59.0 <tibble [1 × 2]>
 4 Angola      Africa    <tibble [12 × 4]>          37.9 <tibble [1 × 2]>
 5 Argentina   Americas  <tibble [12 × 4]>          69.1 <tibble [1 × 2]>
 6 Australia   Oceania   <tibble [12 × 4]>          74.7 <tibble [1 × 2]>
 7 Austria     Europe    <tibble [12 × 4]>          73.1 <tibble [1 × 2]>
 8 Bahrain     Asia      <tibble [12 × 4]>          65.6 <tibble [1 × 2]>
 9 Bangladesh  Asia      <tibble [12 × 4]>          49.8 <tibble [1 × 2]>
10 Belgium     Europe    <tibble [12 × 4]>          73.6 <tibble [1 × 2]>
# … with 132 more rows


What does the new tibbles look like in the `life_exp_ci` column?

In [54]:
by_country$life_exp_ci[[1]]

lower_ci,upper_ci
<dbl>,<dbl>
34.52446,40.12731


### Simplifying the list-column back down to a data frame or atomic vector

After we create some other intermediate list-columns with `map`, we usually want to get some values back as regular atomic vector columns in our data frame, for visualization, further analysis, or reporting. 

We will first demonstrate how to do this using `unnest` in our example to covert the `life_exp_ci` list column to two columns, one for the lower bound of the confidence interval, and one for the upper bound of the confidence interval:

In [55]:
# unnest the ci column
by_country %>% 
    unnest(life_exp_ci) %>% 
    print()

# A tibble: 142 × 6
# Groups:   continent, country [142]
   country     continent data              mean_life_exp lower_ci upper_ci
   <fct>       <fct>     <list>                    <dbl>    <dbl>    <dbl>
 1 Afghanistan Asia      <tibble [12 × 4]>          37.5     34.5     40.1
 2 Albania     Europe    <tibble [12 × 4]>          68.4     64.8     71.7
 3 Algeria     Africa    <tibble [12 × 4]>          59.0     53.4     64.4
 4 Angola      Africa    <tibble [12 × 4]>          37.9     35.7     39.8
 5 Argentina   Americas  <tibble [12 × 4]>          69.1     66.7     71.4
 6 Australia   Oceania   <tibble [12 × 4]>          74.7     72.5     77.0
 7 Austria     Europe    <tibble [12 × 4]>          73.1     70.8     75.4
 8 Bahrain     Asia      <tibble [12 × 4]>          65.6     60.8     70.2
 9 Bangladesh  Asia      <tibble [12 × 4]>          49.8     44.8     54.6
10 Belgium     Europe    <tibble [12 × 4]>          73.6     71.7     75.7
# … with 132 more rows


### The whole shebang:

1. Create a list column using a function such as `nest`

2. Create other intermediate list-columns by transforming existing list columns with `map`

3. Simplify the list-column back down to a data frame or atomic vector using `unnest` & `mutate` + `map_dbl`:

In [56]:
gap_lifeExp_ci <- function(df, statistic) {
  df %>% 
        specify(response = lifeExp) %>% 
        generate(reps = 1000, type = "bootstrap")  %>% 
        calculate(stat = statistic)  %>% 
        get_ci()
}

by_country <- gapminder %>%
    group_by(continent, country) %>%
    nest() %>% 
    mutate(mean_life_exp = map_dbl(data, ~mean(.$lifeExp)), 
    life_exp_ci = map(data, ~gap_lifeExp_ci(., "mean")))
print(by_country)

# A tibble: 142 × 5
# Groups:   continent, country [142]
   country     continent data              mean_life_exp life_exp_ci     
   <fct>       <fct>     <list>                    <dbl> <list>          
 1 Afghanistan Asia      <tibble [12 × 4]>          37.5 <tibble [1 × 2]>
 2 Albania     Europe    <tibble [12 × 4]>          68.4 <tibble [1 × 2]>
 3 Algeria     Africa    <tibble [12 × 4]>          59.0 <tibble [1 × 2]>
 4 Angola      Africa    <tibble [12 × 4]>          37.9 <tibble [1 × 2]>
 5 Argentina   Americas  <tibble [12 × 4]>          69.1 <tibble [1 × 2]>
 6 Australia   Oceania   <tibble [12 × 4]>          74.7 <tibble [1 × 2]>
 7 Austria     Europe    <tibble [12 × 4]>          73.1 <tibble [1 × 2]>
 8 Bahrain     Asia      <tibble [12 × 4]>          65.6 <tibble [1 × 2]>
 9 Bangladesh  Asia      <tibble [12 × 4]>          49.8 <tibble [1 × 2]>
10 Belgium     Europe    <tibble [12 × 4]>          73.6 <tibble [1 × 2]>
# … with 132 more rows


## What did we learn:

- how to write anonymous functions
- how to use {purrr} `map_*` with anonymous functions to add extra arguments
- what are nested data frames
- how to use {tidyr}'s `nest` & `unnest` and {purrr} `map_*` functions to work with data frames to nest, modify and unnest data frames

### Attribution

- [Advanced R](https://adv-r.hadley.nz/) by Hadley Wickham
- [Jenny Bryan's `purrr` tutorial](https://jennybc.github.io/purrr-tutorial/index.html)